In [ ]:
import numpy as np
from matplotlib import pyplot as plt
 
import pandas as pd

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix,ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler ,MinMaxScaler

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Загрузка таблицы

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/данные с акселерометра/dataset_w8_o25.csv', delimiter = ';')

In [ ]:
data.head(10)

,x1_mean,y1_mean,z1_mean,x2_mean,y2_mean,z2_mean,x3_mean,y3_mean,z3_mean,x4_mean,y4_mean,z4_mean,x1_Minmax,y1_Minmax,z1_Minmax,x2_Minmax,y2_Minmax,z2_Minmax,x3_Minmax,y3_Minmax,z3_Minmax,x4_Minmax,y4_Minmax,z4_Minmax,x1_krt,y1_krt,z1_krt,x2_krt,y2_krt,z2_krt,x3_krt,y3_krt,z3_krt,x4_krt,y4_krt,z4_krt,x1_Std,y1_Std,z1_Std,x2_Std,y2_Std,z2_Std,x3_Std,y3_Std,z3_Std,x4_Std,y4_Std,z4_Std,a1_mean,a2_mean,a3_mean,a4_mean,a1_Minmax,a2_Minmax,a3_Minmax,a4_Minmax,a1_krt,a2_krt,a3_krt,a4_krt,a1_Std,a2_Std,a3_Std,a4_Std,class
0,-0.700,97.312,-62.262,-12.962,20.2,-15.1,-12.600,104.3,-89.4,-159.025,-102.625,-142.100,0.0,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.4,0.1,0.2,-3.000,3.143,-1.733,-1.733,-3.0,-3.0,-3.000,-3.0,-3.0,-1.338,-0.667,1.000,0.000,0.033,0.048,0.048,0.0,0.0,0.000,0.0,0.0,0.139,0.043,0.050,115.528,28.356,137.948,236.671,0.138,0.046,0.000,0.202,0.150,-1.733,-3.000,-1.189,0.043,0.022,0.000,0.064,sitting
1,-0.725,97.338,-62.300,-12.938,20.2,-15.1,-12.612,104.3,-89.4,-159.150,-102.600,-142.062,0.1,0.1,0.2,0.1,0.0,0.0,0.1,0.0,0.0,0.5,0.0,0.1,-0.667,-1.733,1.000,-1.733,-3.0,-3.0,3.143,-3.0,-3.0,-0.815,-3.000,-1.733,0.043,0.048,0.050,0.048,0.0,0.0,0.033,0.0,0.0,0.150,0.000,0.048,115.570,28.345,137.949,236.722,0.193,0.046,0.009,0.276,-0.970,-1.733,3.143,-0.605,0.060,0.022,0.003,0.080,sitting
2,-0.750,97.362,-62.338,-12.913,20.2,-15.1,-12.638,104.3,-89.4,-159.275,-102.575,-142.038,0.1,0.1,0.1,0.1,0.0,0.0,0.1,0.0,0.0,0.4,0.1,0.1,-2.000,-1.733,-1.733,3.143,-3.0,-3.0,-1.733,-3.0,-3.0,-1.338,-0.667,-1.733,0.050,0.048,0.048,0.033,0.0,0.0,0.048,0.0,0.0,0.139,0.043,0.048,115.611,28.333,137.951,236.780,0.139,0.046,0.009,0.166,-1.674,3.143,-1.733,-1.316,0.060,0.015,0.004,0.060,sitting
3,-0.775,97.388,-62.362,-12.900,20.2,-15.1,-12.662,104.3,-89.4,-159.412,-102.550,-142.000,0.1,0.1,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.5,0.1,0.2,-0.667,3.143,-1.733,-3.000,-3.0,-3.0,-1.733,-3.0,-3.0,-1.110,-2.000,1.000,0.043,0.033,0.048,0.000,0.0,0.0,0.048,0.0,0.0,0.169,0.050,0.050,115.646,28.328,137.954,236.839,0.139,0.000,0.009,0.233,0.672,-3.000,-1.733,-1.148,0.047,0.000,0.004,0.077,sitting
4,-0.800,97.400,-62.412,-12.900,20.2,-15.1,-12.688,104.3,-89.4,-159.562,-102.525,-141.962,0.0,0.0,0.2,0.0,0.0,0.0,0.1,0.0,0.0,0.5,0.1,0.1,-3.000,-3.000,-0.314,-3.000,-3.0,-3.0,3.143,-3.0,-3.0,-1.461,-0.667,-1.733,0.000,0.000,0.060,0.000,0.0,0.0,0.033,0.0,0.0,0.180,0.043,0.048,115.684,28.328,137.956,236.907,0.108,0.000,0.009,0.234,-0.315,-3.000,3.143,-1.405,0.032,0.000,0.003,0.084,sitting
5,-0.800,97.425,-62.450,-12.900,20.2,-15.1,-12.700,104.3,-89.4,-159.712,-102.475,-141.938,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.1,0.1,-3.000,-0.667,-2.000,-3.000,-3.0,-3.0,-3.000,-3.0,-3.0,-1.089,-0.667,-1.733,0.000,0.043,0.050,0.000,0.0,0.0,0.000,0.0,0.0,0.190,0.043,0.048,115.725,28.328,137.957,236.972,0.138,0.000,0.000,0.301,-1.121,-3.000,-3.000,-0.785,0.057,0.000,0.000,0.090,sitting
6,-0.800,97.450,-62.500,-12.888,20.2,-15.1,-12.700,104.3,-89.4,-159.912,-102.450,-141.888,0.0,0.1,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.7,0.1,0.2,-3.000,-2.000,-1.000,3.143,-3.0,-3.0,-3.000,-3.0,-3.0,-1.021,-2.000,-0.314,0.000,0.050,0.071,0.033,0.0,0.0,0.000,0.0,0.0,0.226,0.050,0.060,115.773,28.322,137.957,237.066,0.192,0.045,0.000,0.309,-1.540,3.143,-3.000,-0.922,0.074,0.015,0.000,0.103,sitting
7,-0.812,97.475,-62.562,-12.862,20.2,-15.1,-12.700,104.3,-89.4,-160.125,-102.412,-141.838,0.1,0.1,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.8,0.2,0.2,3.143,-0.667,-0.739,-1.733,-3.0,-3.0,-3.000,-3.0,-3.0,-1.250,-0.314,-0.739,0.033,0.043,0.070,0.048,0.0,0.0,0.000,0.0,0.0,0.277,0.060,0.070,115.828,28.311,137.957,237.163,0.193,0.045,0.000,0.334,-1.065,-1.733,-3.000,-1.510,0.065,0.022,0.000,0.127,sitting
8,-0.838,97.525,-62.625,-12.838,20.2,-15.1,-12.712,104.3,-89.4,-160.375,-102.362,-141.775,0.1,0.1,0.3,0.1,0.0,0.0,0.1,0.0,0.0,1.0,0.1,0.3,-1.733,-0.667,-0.880,-1.733,-3.0,-3.0,3.143,-3.0,-3.0,-1.149,-1.733,-0.880,0.048,0.043,0.097,0.048,0.0,0.0,0.033,0.0,0.0,0.323,0.048,0.097,115.904,28.299,137.958,237.273,0.247,0.045,0.009,0.454,-0.749,-1.733,3.143,-

Кодирование меток. Деление на тренировочную валидационную и тестовую выборки. Стандартизация всех наборов.

Деление выборки:

* Тренировочная - 60%
* Валидационная - 20%
* Тестовая - 20% 

Из-за дисбаланса классов выбрана метрика F-мера с микро-усреднением. Она позволяет учесть ошибки на малочисленных классах.

In [ ]:
# подготовка меток
le =  LabelEncoder()
le.fit(data['class'])
Y = le.transform(data['class'])

In [ ]:
# подготовка матрицы X и деление на 3 выборки
X = np.array(data[ data.columns[:-1] ])
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.4, random_state=42 ,stratify = Y )
X_val, X_test, Y_val, Y_test = train_test_split(X_val,  Y_val, test_size=0.5, random_state=42 ,stratify =  Y_val )

In [ ]:
# просмотр количества объектов в выборках
print('train set ',X_train.shape[0],' samples')
print('validation set ',X_val.shape[0],' samples')
print('test set ',X_test.shape[0],' samples')

train set  34330  samples
validation set  11444  samples
test set  11444  samples


In [ ]:
# стандартизация столбцов в выборках
scaler_std = StandardScaler().fit(X_train)
X_train_std  = scaler_std.transform(X_train)
X_val_std =  scaler_std.transform(X_val)
X_test_std =  scaler_std.transform(X_test)

Протетируем первую модель - логистическую регрессию.

Подбор параметров (С - коэффициент регуляризации ) softmax регрессии на кросс валидации.

In [ ]:
 
c_vars = [0.5 *i for i in range(1,20)]
gsC =GridSearchCV(estimator= LogisticRegression( penalty ='l2',multi_class='multinomial',solver='newton-cg',verbose=2),
             param_grid= {'C':c_vars },
             scoring ='f1_micro',
             cv = StratifiedKFold(n_splits=5,  shuffle=True, random_state = 42 ) ,
             verbose =4, n_jobs =-1)
gsC.fit(X_train_std, Y_train)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  95 out of  95 | elapsed:  6.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.1s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100,
                                          multi_class='multinomial',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='newton-cg',
                                          tol=0.0001, verbose=2,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0,
                               5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=Fal

In [ ]:
gsC.best_params_

{'C': 9.0}

 Обучим модель и посмотрим ркзльтаты её работы на вылидационной и тестовой выборках

In [ ]:
# model - softmax - std scale - l2
model_softmax_std = LogisticRegression(penalty = 'l2' ,C = gsC.best_params_['C'] , multi_class='multinomial',solver='newton-cg',verbose=2)
model_softmax_std.fit(X_train_std, Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.4s finished


LogisticRegression(C=9.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=2,
                   warm_start=False)

Прогноз модели и матрица ошибок на валидационной выборке.

In [ ]:
Y_val_pred = model_softmax_std.predict(X_val_std)
cm_std = confusion_matrix(le.inverse_transform(Y_val),le.inverse_transform(Y_val_pred ) , labels=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
score_std = f1_score(Y_val,Y_val_pred  , average='micro')
cm_df_std = pd.DataFrame(cm_std,columns=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'], index=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])

In [ ]:
 
cm_df_std

,sitting,sittingdown,standing,standingup,walking
sitting,3561,0,0,1,0
sittingdown,0,830,0,0,2
standing,0,0,3288,0,0
standingup,0,0,0,828,0
walking,0,0,7,0,2927


Значение F1 - метрики

In [ ]:
 score_std

0.9991261796574624

Прогноз модели и матрица ошибок на тестовой выборке.

In [ ]:
Y_test_pred = model_softmax_std.predict(X_test_std)
cm_std = confusion_matrix(le.inverse_transform(Y_test),le.inverse_transform(Y_test_pred ) , labels=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
score_std = f1_score(Y_test,Y_test_pred  , average='micro')
cm_df_std = pd.DataFrame(cm_std,columns=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'], index=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])

In [ ]:
cm_df_std

,sitting,sittingdown,standing,standingup,walking
sitting,3562,0,0,0,0
sittingdown,0,831,1,0,0
standing,0,0,3289,0,0
standingup,0,0,0,828,0
walking,0,0,5,0,2928


Значение F1 - метрики

In [ ]:
score_std

0.9994757077944775

Произведем отбор признаков. 

Возьмем разные сочетания признаков, обучаем модель , подбираем гиперпараметры для неё и посмотрим значения метрик на валидационном наборе. 

Лучшие модели проверим на тестовом наборе.  Посмотрим как работает лучшая модель Softmax регрессии на разный наборах признаков.

Мера качества F-мера с микроусреднением

Рассмотрим следующие варианты поднаборов:
* Среднее и стандартное отклонение по всем координатам, среднее и стандартное отклонение по модулю ускорения
* Среднее по всем координатам, среднее по модулю ускорения
* Среднее по всем координатам
* Среднее по модулю ускорения

In [ ]:
variants =[ 
    ['x1_mean', 'y1_mean', 'z1_mean', 'x2_mean', 'y2_mean', 'z2_mean','x3_mean', 'y3_mean', 'z3_mean', 'x4_mean', 'y4_mean', 'z4_mean', 
    'x1_Std', 'y1_Std', 'z1_Std', 'x2_Std', 'y2_Std', 'z2_Std', 'x3_Std', 'y3_Std', 'z3_Std', 'x4_Std', 'y4_Std', 'z4_Std', 
     'a1_mean','a2_mean', 'a3_mean', 'a4_mean',
    'a1_Std', 'a2_Std', 'a3_Std', 'a4_Std'
       ],

   ['x1_mean', 'y1_mean', 'z1_mean', 'x2_mean', 'y2_mean', 'z2_mean','x3_mean', 'y3_mean', 'z3_mean', 'x4_mean', 'y4_mean', 'z4_mean', 
     'a1_mean','a2_mean', 'a3_mean', 'a4_mean' ],

     ['x1_mean', 'y1_mean', 'z1_mean', 'x2_mean', 'y2_mean', 'z2_mean','x3_mean', 'y3_mean', 'z3_mean', 'x4_mean', 'y4_mean', 'z4_mean'],
      
      ['a1_mean','a2_mean', 'a3_mean', 'a4_mean']

    ]

Вспомогательные функции

In [ ]:
# фукция выполняет решетчатый поиск по заданной сетке, метрике качества, CV-схеме и возвращает 
# модель с лучшими параметрами, сами парамерами и и оценку на кросс-валидации
# model  - модель для оптимизации
# grid_par - сетка параметров
# score_mes - метрика качества
# spliter_cvd - СV - схема
# verbs - параметр verbose
# n_proc - количество потоков процессора
def fit_and_choose(X,Y , model ,  grid_par , score_mes , spliter_cvd , verbs ,n_proc ):
    gsC =GridSearchCV(estimator= model,
             param_grid=   grid_par,
             scoring =score_mes,
             refit=True,
             cv = spliter_cvd ,
             verbose =  verbs,
             n_jobs = n_proc
             )
    gsC.fit(X, Y)
    return {'best model': gsC.best_estimator_ ,
          'best params' : gsC.best_params_,
          'best score': gsC.best_score_}
   

In [ ]:
# функция принимает прогнозы модели (Y_predicted) и правильные прогнозы ( Y_true)
#le - кодировщик для декодирования меток класса
# функция возвращает матрицу ошибок и значение F меры
def conf_matr_and_score_f1(Y_predicted , Y_true,le):

    cmatr  = confusion_matrix(le.inverse_transform(Y_true),le.inverse_transform(Y_predicted) , labels=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
    score_f1 = f1_score(Y_true,Y_predicted  , average='micro')
    cmatr_df = pd.DataFrame(cmatr,columns=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'], index=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
    return {'conf_matr': cmatr_df , 'f1-score':score_f1 }

In [1]:
# функция для отбора признаков из 4 вышеперечисленных вариантов
# возвращает:
# проверенные признаки
# матрицу ошибок на тренировочной выборке
# матрицу ошибок на валидационной выборке
# F меру на тренировочной выборке
# F меру на валидационной выборке
# Использованную модель
# Гиперпараметры модели
'''dt - датафрейм
col_class - колонка с меткой класса
 vars - варианты столбцов из датафрейма
 modl - модель
grid_parm - сетка параметров поиска
score_mesr - мера оценки для кросс - валидации
spliter_cvd - разделитель для кросс-валидации
verbos - вывод информации по процессу на кросс-валидации
n_procs - число потоков'''
def feature_select(dt ,col_class, vars , modl ,  grid_parm , score_mesr , spliter_cvd , verbos ,n_procs ):
    list_of_sel_results = []
    # циклический проход по списку вариантов
    for vari in vars:
        print('checking variant:')
        # вывод списка признаков на печать
        for i in vari:
            print(i,' ',end = '')
        print() 
     
        # кодирование меток 
        le =  LabelEncoder()
        le.fit(dt[col_class])
        Y = le.transform(dt[col_class])
        # разделние выборок в отношении 60-20-20 (train , validation , test)
        X =np.array( dt[vari]) 
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.4, random_state=42 ,stratify = Y )
        X_val, X_test, Y_val, Y_test = train_test_split(X_val,  Y_val, test_size=0.5, random_state=42 ,stratify =  Y_val )
        # масштабирование признаков
        scaler_std = StandardScaler().fit(X_train)
        X_train_scaled_std = scaler_std.transform(X_train)
        X_val_scaled_std =  scaler_std.transform(X_val)
        # оптимизация гиперпараметров модели
        reslt = fit_and_choose(X_train_scaled_std,Y_train , 
                        modl   , 
                       grid_parm,
                        score_mesr , 
                        spliter_cvd,
                         verbos ,
                         n_procs )
        bModel = reslt['best model']
        Y_pred_val = bModel.predict(X_val_scaled_std)
        Y_pred_train = bModel.predict(X_train_scaled_std)
        # получение матриц ошибок и F мер
        cm_train= confusion_matrix(le.inverse_transform(Y_train),le.inverse_transform(Y_pred_train) ,
                               labels=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
        score_train = f1_score(Y_train,Y_pred_train , average='micro')
        cm_df_train = pd.DataFrame(cm_train,columns=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'], 
                               index=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
    
        cm_val= confusion_matrix(le.inverse_transform(Y_val),le.inverse_transform(Y_pred_val) ,
                               labels=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
        score_val = f1_score(Y_val,Y_pred_val , average='micro')
        cm_df_val = pd.DataFrame(cm_val,columns=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'], 
                               index=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
    
        list_of_sel_results.append({'variant': vari,
                                'conf_matr_train':cm_df_train ,
                                'conf_matr_val': cm_df_val ,
                                'f1_train':score_train  ,
                                'f1_val': score_val  ,
                                'model' : bModel,
                                'model_params': reslt['best params']
                                })
        return list_of_sel_results
    

In [ ]:
data_ftr = feature_select(data ,
                          'class', 
                          variants ,
                          LogisticRegression( penalty ='l2',multi_class='multinomial',solver='newton-cg',verbose=2) ,
                          {'C': c_vars } ,
                          'f1_micro'  ,
                          StratifiedKFold(n_splits=10,  shuffle=True, random_state = 42 ) , 
                          4 ,
                          -1 ) 

checking variant:
x1_mean  y1_mean  z1_mean  x2_mean  y2_mean  z2_mean  x3_mean  y3_mean  z3_mean  x4_mean  y4_mean  z4_mean  x1_Std  y1_Std  z1_Std  x2_Std  y2_Std  z2_Std  x3_Std  y3_Std  z3_Std  x4_Std  y4_Std  z4_Std  a1_mean  a2_mean  a3_mean  a4_mean  a1_Std  a2_Std  a3_Std  a4_Std  
Fitting 10 folds for each of 19 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   49.9s
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:  8.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.1s finished


checking variant:
x1_mean  y1_mean  z1_mean  x2_mean  y2_mean  z2_mean  x3_mean  y3_mean  z3_mean  x4_mean  y4_mean  z4_mean  a1_mean  a2_mean  a3_mean  a4_mean  
Fitting 10 folds for each of 19 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   36.3s
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:  6.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s finished


checking variant:
x1_mean  y1_mean  z1_mean  x2_mean  y2_mean  z2_mean  x3_mean  y3_mean  z3_mean  x4_mean  y4_mean  z4_mean  
Fitting 10 folds for each of 19 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:  5.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.0s finished


checking variant:
a1_mean  a2_mean  a3_mean  a4_mean  
Fitting 10 folds for each of 19 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:  3.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s finished


Вывод результатов по вариантам признаков

In [ ]:
data_ftr[0]

{'conf_matr_train':              sitting  sittingdown  standing  standingup  walking
 sitting        10685            0         0           0        0
 sittingdown        0         2496         0           0        0
 standing           0            0      9865           0        0
 standingup         0            0         0        2484        0
 walking            0            0        15           0     8785,
 'conf_matr_val':              sitting  sittingdown  standing  standingup  walking
 sitting         3562            0         0           0        0
 sittingdown        0          832         0           0        0
 standing           0            0      3288           0        0
 standingup         0            0         0         828        0
 walking            0            0         7           0     2927,
 'f1_train': 0.999563064375182,
 'f1_val': 0.9993883257602237,
 'model': LogisticRegression(C=9.5, class_weight=None, dual=False, fit_intercept=True,
                    

In [ ]:
data_ftr[1]

{'conf_matr_train':              sitting  sittingdown  standing  standingup  walking
 sitting        10685            0         0           0        0
 sittingdown        1         2477        15           1        2
 standing           0            0      9863           2        0
 standingup         0            5         0        2477        2
 walking            0            6        40           0     8754,
 'conf_matr_val':              sitting  sittingdown  standing  standingup  walking
 sitting         3562            0         0           0        0
 sittingdown        2          824         6           0        0
 standing           0            0      3287           1        0
 standingup         0            4         0         823        1
 walking            0            0        15           0     2919,
 'f1_train': 0.9978444509175648,
 'f1_val': 0.997465921006641,
 'model': LogisticRegression(C=9.5, class_weight=None, dual=False, fit_intercept=True,
                    

In [ ]:
data_ftr[2]

{'conf_matr_train':              sitting  sittingdown  standing  standingup  walking
 sitting        10683            0         0           2        0
 sittingdown        1         2470        22           2        1
 standing           0            0      9857           0        8
 standingup         0           10         0        2471        3
 walking            0            8        56           0     8736,
 'conf_matr_val':              sitting  sittingdown  standing  standingup  walking
 sitting         3561            0         0           1        0
 sittingdown        2          823         7           0        0
 standing           0            0      3288           0        0
 standingup         0            5         0         821        2
 walking            0            0        23           0     2911,
 'f1_train': 0.9967084182930381,
 'f1_val': 0.9965047186298497,
 'model': LogisticRegression(C=9.5, class_weight=None, dual=False, fit_intercept=True,
                   

In [ ]:
data_ftr[3]

{'conf_matr_train':              sitting  sittingdown  standing  standingup  walking
 sitting        10509          154         7           6        9
 sittingdown      150         1840       282         162       62
 standing         299            0      7729         373     1464
 standingup         3          412       662        1373       34
 walking            0            0      3000           0     5800,
 'conf_matr_val':              sitting  sittingdown  standing  standingup  walking
 sitting         3491           67         2           0        2
 sittingdown       60          613        97          41       21
 standing          96            0      2622         121      449
 standingup         1          151       200         467        9
 walking            0            0       981           0     1953,
 'f1_train': 0.7937955141275852,
 'f1_val': 0.7991960852848654,
 'model': LogisticRegression(C=9.0, class_weight=None, dual=False, fit_intercept=True,
                   

Две последние модели проверим на тестовом наборе

Пространство из 12 признаков

Подготовка данных и оценка модели на тестовом наборе

In [ ]:
 
X = np.array(data[ ['x1_mean', 'y1_mean', 'z1_mean', 'x2_mean', 'y2_mean', 'z2_mean','x3_mean', 'y3_mean', 'z3_mean', 'x4_mean', 'y4_mean', 'z4_mean'] ])
X_train12, X_val12, Y_train12, Y_val12 = train_test_split(X, Y, test_size=0.4, random_state=42 ,stratify = Y )
X_val12, X_test12, Y_val12, Y_test12 = train_test_split(X_val12,  Y_val12, test_size=0.5, random_state=42 ,stratify =  Y_val12 )

In [ ]:
 
le =  LabelEncoder()
le.fit(data['class'])
Y = le.transform(data['class'])

In [ ]:
 
scaler_std12 = StandardScaler().fit(X_train12)
X_train_std12 = scaler_std12.transform(X_train12)
X_val_std12 =  scaler_std12.transform(X_val12)
X_test_std12 =  scaler_std12.transform(X_test12)

In [ ]:
Y_test_pred12 = data_ftr[2]['model'].predict(X_test_std12)
 
q12 = conf_matr_and_score_f1(Y_test_pred12  ,Y_test12,le)

Матрица ошибок

In [ ]:
q12['conf_matr']

,sitting,sittingdown,standing,standingup,walking
sitting,3560,0,0,2,0
sittingdown,3,827,2,0,0
standing,0,0,3282,0,7
standingup,0,4,0,823,1
walking,0,2,14,0,2917


F мера

In [ ]:
q12['f1-score']

0.9969416288011185

Пространство из 4 признаков

Подготовка данных и оценка модели на тестовом наборе

In [ ]:
 
X = np.array(data[  ['a1_mean','a2_mean', 'a3_mean', 'a4_mean'] ])
X_train4, X_val4, Y_train4, Y_val4 = train_test_split(X, Y, test_size=0.4, random_state=42 ,stratify = Y )
X_val4, X_test4, Y_val4, Y_test4 = train_test_split(X_val4,  Y_val4, test_size=0.5, random_state=42 ,stratify =  Y_val4 )
 
le =  LabelEncoder()
le.fit(data['class'])
Y = le.transform(data['class'])
 
scaler_std4 = StandardScaler().fit(X_train4)
X_train_std4 = scaler_std4.transform(X_train4)
X_val_std4 =  scaler_std4.transform(X_val4)
X_test_std4 =  scaler_std4.transform(X_test4)

 


In [ ]:
Y_test_pred4 = data_ftr[3]['model'].predict(X_test_std4)
q4 = conf_matr_and_score_f1(Y_test_pred4  ,Y_test4,le)

Матрица ошибок

In [ ]:
q4['conf_matr']

,sitting,sittingdown,standing,standingup,walking
sitting,3508,53,0,1,0
sittingdown,46,614,107,46,19
standing,114,0,2608,110,457
standingup,1,133,223,463,8
walking,0,0,929,0,2004


F - мера

In [ ]:
q4['f1-score']   

0.803652569031807

Выводы: можно строить модели с этими 12 и 4 признаками. Результат на тестовой выборке - вполне хороший. Это говорит о том что в этих пространствах признаков данные достаточно хорошо линейно разделимы.